<a href="https://colab.research.google.com/github/psaboia/CV2_project/blob/main/maskRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Trying hyperparameters CV2 - T6]
#**Occluded Object Detection with Scarce Data**

## **MaskRCNN Experiment**

Summary
-------

1.   Data loading
  1.   Setting up infra
  2.   Setting up the custom dataset
2.   Trainnig
  1.   Setting up Trainnig
  2.   Performing trainnig
  3.   Saving fine-tuned model
3.   Evaluation & Inference (using the saved fine-tuned model)
  1.   Validation set
      1.   Evaluation using AP metric
      2.   Inference
  2.   Test set
      1.   Inference
      2.   Evaluation using AP metric
            *   **Light**: ambient, **View**: side
            *   **Light**: ambient, **View**: top
            *   **Light**: side, **View**: side
            *   **Light**: side, **View**: top
            *   **Light**: top, **View**: side
            *   **Light**: top, **View**: top
  3.   Saving results


# Data Loading

## Setting up infra

In [ ]:
path_exp = '/content/drive/MyDrive/CSE/Spring21/cv2/task_2/models/exp3/'


In [ ]:
## Step 1: Install Detectron2
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

# See https://detectron2.readthedocs.io/tutorials/install.html for more instructions, if needed
import torch
assert torch.__version__.startswith("1.8") # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

!pip install pillow==4.1.1
%reload_ext autoreload
%autoreload
!pip install PIL
!pip install image

# Loading Detectron2 and other basic libraries, used later
import tensorflow as tf
tf.test.gpu_device_name()

# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# Import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

import numpy as np
def concat_imgs(im1,im2):
  [h,w,d] = im1.shape
  comp_im = np.ndarray(shape=(h,2*w,d), dtype=im1.dtype)
  comp_im[:,:w,:] = im1
  comp_im[:,w:,:] = im2
  return comp_im

## Setting up the custom dataset

In [ ]:
## Step 2: Setting up dataset
from os.path import isfile, isdir, join
from google.colab import drive
drive.mount('/content/drive')

# Prix dataset
prix_dataset_train = "/content/drive/MyDrive/CSE/Spring21/cv2/dataset/prix_objects_v3"

prix_dataset_val = "/content/drive/MyDrive/CSE/Spring21/cv2/dataset/prix_objects_v1/val"

from detectron2.data.datasets import register_coco_instances
register_coco_instances("prix_train", {}, join(prix_dataset_train,"annotations.json"), prix_dataset_train)
register_coco_instances("prix_val", {}, join(prix_dataset_val,"annotations.json"), prix_dataset_val)

To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set

In [ ]:
import random
from detectron2.utils.visualizer import Visualizer
import cv2

from detectron2.data import DatasetCatalog, MetadataCatalog

dataset_dicts = DatasetCatalog.get("prix_train")
prix_metadata = MetadataCatalog.get("prix_train")

for d in random.sample(dataset_dicts, 10):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=prix_metadata, scale=0.05)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

# Trainnig

## Setting up trainnig

Creating a hook to compute our validation loss every few iterations

In [ ]:
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader, build_detection_train_loader
from detectron2.engine import DefaultTrainer
import detectron2.utils.comm as comm
from detectron2.data import transforms as T
from detectron2.data import detection_utils as utils
from detectron2.evaluation import COCOEvaluator
import logging
import torch
import time
import datetime
import copy



class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            

            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()

            if torch.cuda.is_available():
                torch.cuda.synchronize()

            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start

            if idx >= num_warmup * 2 or seconds_per_img > 5:

                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
                
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses


    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced


    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)



class MyTrainer(DefaultTrainer):


    @classmethod
    def build_train_loader(cls, cfg):
        print("AQIOOOOOOOOOO ")        
        train_augmentations = [
            T.RandomBrightness(0.5, 2),
            T.RandomContrast(0.5, 2),
            T.RandomSaturation(0.5, 2),
            T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
            T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
            T.RandomRotation(45, expand=False),
            T.Resize((300, 300)),

        ]        
        return  build_detection_train_loader(cfg,mapper=DatasetMapper(cfg, is_train=True, augmentations=train_augmentations))

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)

                     
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1, LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks       




## Performing training

In [ ]:
import os
import numpy as np
from detectron2.structures import BoxMode
from detectron2.config import get_cfg


selectedModel = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"

cfg = get_cfg()
preTrainedModel = selectedModel
cfg.merge_from_file(model_zoo.get_config_file(preTrainedModel))

# training
cfg.DATASETS.TRAIN = ("prix_train",)

# validation
cfg.DATASETS.TEST = ("prix_val", )
cfg.TEST.EVAL_PERIOD = 100
cfg.DATALOADER.NUM_WORKERS = 4
 
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(preTrainedModel)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64   # (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
 
# Solver options
cfg.SOLVER.BASE_LR = 1e-3           # Base learning rate
cfg.SOLVER.GAMMA = 0.5              # Learning rate decay
cfg.SOLVER.STEPS = (250, 500, 750)  # Iterations at which to decay learning rate
cfg.SOLVER.MAX_ITER = 1000          # Maximum number of iterations
cfg.SOLVER.WARMUP_ITERS = 100       # Warmup iterations to linearly ramp learning rate from zero # adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.IMS_PER_BATCH = 1        # Lower to reduce memory usage (1 is the lowest)



os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg)     # will use the Detectron2 default trainer
trainer = MyTrainer(cfg)            # will use our custom trainer that computes the validation losses too
trainer.resume_or_load(resume=False)
trainer.train()


%load_ext tensorboard
%tensorboard --logdir output


## Saving fine-tuned model

In [ ]:
# Save model on my drive
path_exp = '/content/drive/MyDrive/CSE/Spring21/cv2/task_2/models/exp3/'
!cp -r output/ $path_exp


# Evaluation & Inference (using the saved fine-tuned model)

---
## Validation set (only objects)
---

### **Evaluation** using AP metric

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset


cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8   # set the testing threshold for this model
cfg.DATASETS.TEST = ("prix_val", )
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("prix_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "prix_val")
inference_on_dataset(trainer.model, val_loader, evaluator)

### **Inference**
visualizing the prediction results on samples from the prix validation set

In [ ]:
import numpy as np
def concat_imgs(im1,im2):
  [h,w,d] = im1.shape
  comp_im = np.ndarray(shape=(h,2*w,d), dtype=im1.dtype)
  comp_im[:,:w,:] = im1
  comp_im[:,w:,:] = im2
  return comp_im
  
# Inference
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import Visualizer

dataset_name = "prix_val"
num_samples = 20

for d in random.sample(DatasetCatalog.get(dataset_name), num_samples):        
  im = cv2.imread(d["file_name"])
  outputs = predictor(im)
      
  # groundtruth image
  visualizer = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get(dataset_name), scale=0.05)
  vis_gt = visualizer.draw_dataset_dict(d)      
  im_gt = vis_gt.get_image()[:, :, ::-1]
 
  # predicted image
  visualizer = Visualizer(im[:, :, ::-1], metadata=prix_metadata, scale=0.05)
  vis_pr = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))            
  im_pr = vis_pr.get_image()[:, :, ::-1]
      
  # show images
  print(dataset_name + ' | ' + d["file_name"].split('/')[-1])
  cv2_imshow(concat_imgs(im_pr, im_gt))

---
## Test set ( Uncluttered Tote Images)
---

### **Inference**
*Visualizing the prediction results on samples from the prix_uncluttered_tote unseen set*

In [ ]:
# Import some common libraries
from os import listdir
from os.path import isfile, join
import random
import re
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

import numpy as np
def concat_imgs(im1,im2):
  [h,w,d] = im1.shape
  comp_im = np.ndarray(shape=(h,2*w,d), dtype=im1.dtype)
  comp_im[:,:w,:] = im1
  comp_im[:,w:,:] = im2
  return comp_im

In [ ]:
## Step 2: Setting up dataset
from os.path import isfile, isdir, join
from google.colab import drive
drive.mount('/content/drive')

*Register coco instances => Tote images*

In [ ]:
# Register coco instances
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

path_test_dataset = "/content/drive/MyDrive/CSE/Spring21/cv2/dataset/prix_cluttered_objects_labelme"
list_dataset_name = listdir(path_test_dataset)

for d in list_dataset_name:
  register_coco_instances(d, {}, join(join(path_test_dataset, d),"annotations.json"), join(path_test_dataset, d))



*Load Model*

In [ ]:
# Load Model
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

# path_exp = '/content/drive/MyDrive/CSE/Spring21/cv2/Task_1/models/post_midterm_model/exp3/'
path_exp = '/content/drive/MyDrive/CSE/Spring21/cv2/task_2/models/exp3/'

path_model = join(path_exp, 'output')
cfg = get_cfg() 
cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'))
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1
#cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.1 # 0.001# 0.01

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST =  0.001# 0.01

cfg.MODEL.WEIGHTS = join(path_model, "model_final.pth")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
predictor = DefaultPredictor(cfg)

*Inference*

In [ ]:
# Inference
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import Visualizer

for dataset_name in list_dataset_name:
  cfg.DATASETS.TEST = (dataset_name, )
  dataset_dicts = DatasetCatalog.get(dataset_name)
  tote_metadata = MetadataCatalog.get(dataset_name)

  print(dataset_name)
  n=10
  scale = 0.3
  for d in random.sample(dataset_dicts, n):        
      im = cv2.imread(d["file_name"])
      outputs = predictor(im)
      
      # groundtruth image
      visualizer = Visualizer(im[:, :, ::-1], metadata=tote_metadata, scale=scale)
      vis_gt = visualizer.draw_dataset_dict(d)      
      im_gt = vis_gt.get_image()[:, :, ::-1]
 
      # predicted image
      visualizer = Visualizer(im[:, :, ::-1], metadata=tote_metadata, scale=scale)
      vis_pr = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))            
      im_pr = vis_pr.get_image()[:, :, ::-1]
      
      # show images
      print(dataset_name + ' | ' + d["file_name"].split('/')[-1])
      cv2_imshow(concat_imgs(im_pr, im_gt))

In [ ]:
# Inference
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import Visualizer

print(dataset_name)
n=5
scale = 0.05
for d in random.sample(dataset_dicts, n):        
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)

    for k in range(10):
      print(k, tote_metadata.thing_classes[k])
      category_k_detections = instances[instances.pred_classes == k]
        
        ## FILTERS
        # max score

      cv2_imshow(im)
      if len(category_k_detections) > 0:
        confident_detection = category_k_detections[category_k_detections.scores == torch.max(category_k_detections.scores)]

          # sort by score
          # _, indices = torch.sort(category_k_detections.scores, descending=True)
          # confident_detections = category_k_detections[indices]

          # only highest than 's'
          # s = 0.9
          # confident_detections = category_k_detections[category_k_detections.scores > s]

        visualizer = Visualizer(im[:, :, ::-1], metadata=tote_metadata, scale=scale)
        vis_pr = visualizer.draw_instance_predictions(category_k_detections.to("cpu"))            
        x = vis_pr.get_image()[:, :, ::-1]


        visualizer = Visualizer(im[:, :, ::-1], metadata=tote_metadata, scale=scale)
        vis_pr = visualizer.draw_instance_predictions(confident_detection.to("cpu"))            
        y = vis_pr.get_image()[:, :, ::-1]

        cv2_imshow(concat_imgs(x, y))

      else:
        print("No instance detected.")  
        

### **Evaluation** using AP metric

In [ ]:
# Register coco instances
from detectron2.data.datasets import register_coco_instances

prix_dataset_test1 = "/content/drive/MyDrive/CSE/Spring21/cv2/dataset/prix_objects_v1/test/light_ambient_vs"
prix_dataset_test2 = "/content/drive/MyDrive/CSE/Spring21/cv2/dataset/prix_objects_v1/test/light_ambient_vt"
prix_dataset_test3 = "/content/drive/MyDrive/CSE/Spring21/cv2/dataset/prix_objects_v1/test/light_side_vs"
prix_dataset_test4 = "/content/drive/MyDrive/CSE/Spring21/cv2/dataset/prix_objects_v1/test/light_side_vt"
prix_dataset_test5 = "/content/drive/MyDrive/CSE/Spring21/cv2/dataset/prix_objects_v1/test/light_top_vs"
prix_dataset_test6 = "/content/drive/MyDrive/CSE/Spring21/cv2/dataset/prix_objects_v1/test/light_top_vt"

register_coco_instances("prix_test_1", {}, join(prix_dataset_test1,"annotations.json"), prix_dataset_test1)
register_coco_instances("prix_test_2", {}, join(prix_dataset_test2,"annotations.json"), prix_dataset_test2)
register_coco_instances("prix_test_3", {}, join(prix_dataset_test3,"annotations.json"), prix_dataset_test3)
register_coco_instances("prix_test_4", {}, join(prix_dataset_test4,"annotations.json"), prix_dataset_test4)
register_coco_instances("prix_test_5", {}, join(prix_dataset_test5,"annotations.json"), prix_dataset_test5)
register_coco_instances("prix_test_6", {}, join(prix_dataset_test6,"annotations.json"), prix_dataset_test6)


In [ ]:
# Load Model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2   # set the testing threshold for this model

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

---
#### **Light**: ambient, **View**: side
---


In [ ]:
name_set = "prix_test_1"
cfg.DATASETS.TEST = (name_set, )

evaluator = COCOEvaluator(name_set, cfg, False, output_dir="./output/"+name_set)
val_loader = build_detection_test_loader(cfg, name_set)
inference_on_dataset(trainer.model, val_loader, evaluator)

#predictor = DefaultPredictor(cfg)


---
#### **Light**: ambient, **View**: top
---


In [ ]:

name_set = "prix_test_2"
cfg.DATASETS.TEST = (name_set, )

evaluator = COCOEvaluator(name_set, cfg, False, output_dir="./output/"+name_set)
val_loader = build_detection_test_loader(cfg, name_set)
inference_on_dataset(trainer.model, val_loader, evaluator)

#predictor = DefaultPredictor(cfg)

--- 
#### **Light**: side , **View**: side
---

In [ ]:
name_set = "prix_test_3"
cfg.DATASETS.TEST = (name_set, )

evaluator = COCOEvaluator(name_set, cfg, False, output_dir="./output/"+name_set)
val_loader = build_detection_test_loader(cfg, name_set)
inference_on_dataset(trainer.model, val_loader, evaluator)

#predictor = DefaultPredictor(cfg)

---
#### **Light**: side, **View**: top
---

In [ ]:
name_set = "prix_test_4"
cfg.DATASETS.TEST = (name_set, )

evaluator = COCOEvaluator(name_set, cfg, False, output_dir="./output/"+name_set)
val_loader = build_detection_test_loader(cfg, name_set)
inference_on_dataset(trainer.model, val_loader, evaluator)

---
#### **Light**: top, **View**: side
---



In [ ]:
name_set = "prix_test_5"
cfg.DATASETS.TEST = (name_set, )

evaluator = COCOEvaluator(name_set, cfg, False, output_dir="./output/"+name_set)
val_loader = build_detection_test_loader(cfg, name_set)
inference_on_dataset(trainer.model, val_loader, evaluator)

#predictor = DefaultPredictor(cfg)

---
#### **Light**: top, **View**: top
---

In [ ]:
name_set = "prix_test_6"
cfg.DATASETS.TEST = (name_set, )

evaluator = COCOEvaluator(name_set, cfg, False, output_dir="./output/"+name_set)
val_loader = build_detection_test_loader(cfg, name_set)
inference_on_dataset(trainer.model, val_loader, evaluator)

# predictor = DefaultPredictor(cfg)

 ### Saving results 

In [ ]:
# Save all results
!cp -r output/ $path_exp